In [1]:
using Flux

In [2]:
using FastAI, FastTimeSeries

# TimeSeries Classification

In [3]:
data, blocks = load(datarecipes()["ecg5000"]);

`getobs` gets us a sample from the TimeSeriesDataset. It returns a tuple with the input time series and the correspodning label.

In [4]:
input, class = sample = getobs(data, 25)

(Float32[-0.28834122 -2.2725453 … 1.722784 1.2959242], "1")

Now we create a learning task for time-series classification. This means using the time-series to predict labels. We will use the `TimeSeriesRow` block as input and `Label` block as the target.

In [5]:
task = SupervisedTask(
    blocks,
    (
        OneHot(),
        setup(TSPreprocessing, blocks[1], data[1].table)
    )
)

SupervisedTask(TimeSeriesRow -> Label{SubString{String}})

The encodings passed in transform samples into formats suitable as inputs and outputs for a model

Let's check that samples from the created data container conform to the blocks of the learning task:

In [6]:
checkblock(task.blocks.sample, sample)

true

To get an overview of the learning task created, and as a sanity test, we can use describetask. This shows us what encodings will be applied to which blocks, and how the predicted ŷ values are decoded.

In [7]:
describetask(task)

**`SupervisedTask` summary**

Learning task for the supervised task with input `TimeSeriesRow` and target `Label{SubString{String}}`. Compatible with `model`s that take in `TimeSeriesRow` and output `OneHotLabel{SubString{String}}`.

Encoding a sample (`encodesample(task, context, sample)`) is done through the following encodings:

|          Encoding |              Name |      `blocks.input` |                      `blocks.target` |
| -----------------:| -----------------:| -------------------:| ------------------------------------:|
|                   | `(input, target)` |     `TimeSeriesRow` |           `Label{SubString{String}}` |
|          `OneHot` |                   |                     | **`OneHotLabel{SubString{String}}`** |
| `TSPreprocessing` |          `(x, y)` | **`TimeSeriesRow`** |                                      |


In [8]:
encoded_sample = encodesample(task, Training(), sample)

(Float32[-0.28937635 -2.2807038 … 1.7289687 1.3005764], Bool[1, 0, 0, 0, 0])

### Visualization Tools for TimeSeries

In [9]:
sample = getobs(data, 1)

(Float32[-0.11252183 -2.8272038 … 0.92528623 0.19313742], "1")

In [ ]:
showsample(task, sample)

In [ ]:
showblock(blocks[1], sample[1])

### Training

We will use a StackedLSTM as a backbone model, and a Dense layer at the front for classification. `taskmodel` knows how to do this by looking at the datablocks used. 

In [31]:
backbone = FastTimeSeries.Models.StackedLSTM(1, 16, 10, 2);

In [32]:
model = FastAI.taskmodel(task, backbone)

Chain(
  FastTimeSeries.Models.tabular2rnn,
  StackedLSTMCell(
    Chain(
      Recur(
        LSTMCell(1 => 10),              # 500 parameters
      ),
      Recur(
        LSTMCell(10 => 16),             # 1_760 parameters
      ),
    ),
  ),
  identity,
  Dense(16 => 5),                       # 85 parameters
)         # Total: 12 trainable arrays, 2_345 parameters,
          # plus 4 non-trainable, 1_664 parameters, summarysize 16.660 KiB.

We can `tasklossfn` to get a loss function suitable for our task.

In [33]:
lossfn = tasklossfn(task)

logitcrossentropy (generic function with 1 method)

Next we create a pair of training and validation data loaders. They take care of batching and loading the data in parallel in the background.

In [34]:
traindl, validdl = taskdataloaders(data, task, 16);

We will use an `Adam` optimzer for this task.

In [35]:
optimizer = ADAM(0.002)

Adam(0.002, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

With the addition of an optimizer and a loss function, we can now create a Learner and start training. 

In [36]:
learner = Learner(model, (traindl, validdl), optimizer, lossfn, ToGPU())

Learner()

In [37]:
fitonecycle!(learner, 10, 0.002)

Epoch 1 TrainingPhase(): 100%|██████████████████████████| Time: 0:00:01


┌───────────────┬───────┬─────────┐
│         Phase │ Epoch │    Loss │
├───────────────┼───────┼─────────┤
│ TrainingPhase │   1.0 │ 1.05207 │
└───────────────┴───────┴─────────┘


Epoch 1 ValidationPhase(): 100%|████████████████████████| Time: 0:00:00


┌─────────────────┬───────┬─────────┐
│           Phase │ Epoch │    Loss │
├─────────────────┼───────┼─────────┤
│ ValidationPhase │   1.0 │ 0.62447 │
└─────────────────┴───────┴─────────┘


Epoch 2 TrainingPhase(): 100%|██████████████████████████| Time: 0:00:02


...

┌─────────────────┬───────┬─────────┐
│           Phase │ Epoch │    Loss │
├─────────────────┼───────┼─────────┤
│ ValidationPhase │   9.0 │ 0.29944 │
└─────────────────┴───────┴─────────┘


Epoch 10 TrainingPhase(): 100%|█████████████████████████| Time: 0:00:02


┌───────────────┬───────┬─────────┐
│         Phase │ Epoch │    Loss │
├───────────────┼───────┼─────────┤
│ TrainingPhase │  10.0 │ 0.26081 │
└───────────────┴───────┴─────────┘


Epoch 10 ValidationPhase(): 100%|███████████████████████| Time: 0:00:00


┌─────────────────┬───────┬─────────┐
│           Phase │ Epoch │    Loss │
├─────────────────┼───────┼─────────┤
│ ValidationPhase │  10.0 │ 0.28765 │
└─────────────────┴───────┴─────────┘


We can save the model for later inference using `savetaskmodel`:

In [ ]:
savetaskmodel("tsclassification.jld2", task, learner.model)